In [82]:
from lib.dataset.mesh_util import *
#from lib.smplx.body_models import SMPLX
from lib.dataset.TestDataset import TestDataset
from lib.common.config import cfg
from lib.pymafx.core import path_config
import numpy as np

In [83]:
SMPLX_object = SMPLX()

In [84]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

Type of each parameter in 'dataset_param':

    <class 'str'>
    <class 'NoneType'>
    <class 'bool'>
    <class 'str'>
    <class 'int'>
    <class 'bool'>

In [85]:
dataset_param = {
        "image_dir": './examples/testimg/',
        "seg_dir": None,
        "use_seg": True,    # w/ or w/o segmentation
        "hps_type": 'pymafx' ,   # pymafx/pixie
        "vol_res": cfg.vol_res,
        "single": False, #args.multi,
    }


In [86]:
path_config.pymafx_data_dir = "./data/HPS/pymafx_data"

os.path.exists(path_config.pymafx_data_dir)

True

In [87]:
param_npz = np.load('./00006_0.npz' , allow_pickle=True)
print(type(param_npz))
for item in param_npz:
    print (item)

<class 'numpy.lib.npyio.NpzFile'>
global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [88]:
param = {key: torch.as_tensor(param_npz[key]) for key in param_npz.keys()}

In [89]:
for item in param_npz:
    print (item)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [90]:

for key in param.keys():
    print(key)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [91]:
from apps.infer import view_dict_items

In [92]:
view_dict_items(param)

global_orient: torch.Size([1, 3])
body_pose: torch.Size([21, 3])
left_hand_pose: torch.Size([15, 3])
right_hand_pose: torch.Size([15, 3])
jaw_pose: torch.Size([1, 3])
leye_pose: torch.Size([1, 3])
reye_pose: torch.Size([1, 3])
betas: torch.Size([1, 10])
expression: torch.Size([1, 10])
transl: torch.Size([1, 3])


In [93]:
#SMPLX_object = SMPLX('00006_0.npz')

In [96]:
data = np.load('./pymafx_results/econ/obj/300_smpl_00.npy',allow_pickle=True)
#print(data)
data= data.tolist()
for item in data:
    print(item,': ',data[item].shape)

print(data['scale'])

betas :  torch.Size([1, 200])
body_pose :  torch.Size([1, 21, 3])
global_orient :  torch.Size([1, 1, 3])
transl :  torch.Size([1, 3])
expression :  torch.Size([1, 50])
jaw_pose :  torch.Size([1, 1, 3])
left_hand_pose :  torch.Size([1, 15, 3])
right_hand_pose :  torch.Size([1, 15, 3])
scale :  torch.Size([1, 1])
tensor([[1.0443]])


In [95]:
smpl_mesh,smpl_joint = load_fit_body(fitted_path='./results/econ/obj/000023_smpl_00.npy',scale=1)

AttributeError: 'numpy.ndarray' object has no attribute 'keys'